# Upload and deploy a model

In [34]:
import os
import numpy as np
import time

import tensorflow as tf

from google.cloud import aiplatform as vertex_ai
from PIL import Image

tf.get_logger().setLevel('ERROR')

## Configure environment constants


In [2]:
PROJECT = 'jk-mlops-dev'
REGION = 'us-central1'
STAGING_BUCKET = 'gs://jk-vertex-workshop-bucket'

## Intialize Vertex SDK

In [3]:
vertex_ai.init(
    project=PROJECT,
    location=REGION,
    staging_bucket=STAGING_BUCKET
)

## Upload the model

In [10]:
saved_model_path = f'{STAGING_BUCKET}/models/cifar/model'

!saved_model_cli show --dir {saved_model_path} --tag_set serve --signature_def serving_default

2021-06-02 03:01:59.958226: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
The given SavedModel SignatureDef contains the following input(s):
  inputs['conv2d_input'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 32, 32, 3)
      name: serving_default_conv2d_input:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['dense'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 10)
      name: StatefulPartitionedCall:0
Method name is: tensorflow/serving/predict


In [31]:
display_name = 'cifar10-classifier'
serving_image_uri = 'gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-1:latest'

model = vertex_ai.Model.upload(
    display_name=display_name,
    artifact_uri=saved_model_path,
    serving_container_image_uri=serving_image_uri
)

model.wait()

INFO:google.cloud.aiplatform.models:Creating Model
INFO:google.cloud.aiplatform.models:Create Model backing LRO: projects/895222332033/locations/us-central1/models/8137986934472441856/operations/7927636616183545856
INFO:google.cloud.aiplatform.models:Model created. Resource name: projects/895222332033/locations/us-central1/models/8137986934472441856
INFO:google.cloud.aiplatform.models:To use this Model in another session:
INFO:google.cloud.aiplatform.models:model = aiplatform.Model('projects/895222332033/locations/us-central1/models/8137986934472441856')


## Create endpoint

In [22]:
display_name = 'cifar10-endpoint'

endpoint = vertex_ai.Endpoint.create(display_name=display_name)
endpoint

INFO:google.cloud.aiplatform.models:Creating Endpoint
INFO:google.cloud.aiplatform.models:Create Endpoint backing LRO: projects/895222332033/locations/us-central1/endpoints/6900130355643154432/operations/5931416081351573504
INFO:google.cloud.aiplatform.models:Endpoint created. Resource name: projects/895222332033/locations/us-central1/endpoints/6900130355643154432
INFO:google.cloud.aiplatform.models:To use this Endpoint in another session:
INFO:google.cloud.aiplatform.models:endpoint = aiplatform.Endpoint('projects/895222332033/locations/us-central1/endpoints/6900130355643154432')


resource name: projects/895222332033/locations/us-central1/endpoints/6900130355643154432

## Deploy model

In [32]:
for model in vertex_ai.Model.list(filter='display_name="cifar10-classifier"'):
    print(model)

resource name: projects/895222332033/locations/us-central1/models/8137986934472441856


In [33]:
deployed_model_display_name = 'cifar10-v1'
traffic_percentage = 100
machine_type = 'n1-standard-4'

endpoint = model.deploy(
        endpoint=endpoint,
        deployed_model_display_name=deployed_model_display_name,
        machine_type=machine_type,
        traffic_percentage=traffic_percentage)


INFO:google.cloud.aiplatform.models:Deploying model to Endpoint : projects/895222332033/locations/us-central1/endpoints/6900130355643154432
INFO:google.cloud.aiplatform.models:Deploy Endpoint model backing LRO: projects/895222332033/locations/us-central1/endpoints/6900130355643154432/operations/4960890361653231616
INFO:google.cloud.aiplatform.models:Endpoint model deployed. Resource name: projects/895222332033/locations/us-central1/endpoints/6900130355643154432


resource name: projects/895222332033/locations/us-central1/endpoints/6900130355643154432

## Test the model

### Prepare test examples

In [36]:
IMAGE_DIRECTORY = "cifar_test_images"

image_files = [file for file in os.listdir(IMAGE_DIRECTORY) if file.endswith(".jpg")]

# Decode JPEG images into numpy arrays
image_data = [
    np.asarray(Image.open(os.path.join(IMAGE_DIRECTORY, file))) for file in image_files
]

# Scale and convert to expected format
x_test = [(image / 255.0).astype(np.float32).tolist() for image in image_data]

# Extract labels from image name
y_test = [int(file.split("_")[1]) for file in image_files]

### Get the endpoint

In [41]:
for endpoint_info in vertex_ai.Endpoint.list(filter='display_name="cifar10-endpoint"'):
    print(endpoint_info)

resource name: projects/895222332033/locations/us-central1/endpoints/6900130355643154432


In [43]:
endpoint = vertex_ai.Endpoint(endpoint_info.resource_name)

### Call the endpoint

In [44]:
predictions = endpoint.predict(instances=x_test)
y_predicted = np.argmax(predictions.predictions, axis=1)

correct = sum(y_predicted == np.array(y_test))
accuracy = len(y_predicted)
print(
    f"Correct predictions = {correct}, Total predictions = {accuracy}, Accuracy = {correct/accuracy}"
)

Correct predictions = 4, Total predictions = 10, Accuracy = 0.4
